In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse, r2_score
import shap
from sklearn.preprocessing import MinMaxScaler
from kan_p import *
import nlopt
from sklearn.model_selection import cross_val_score
import random
random.seed(50)
import warnings
warnings.filterwarnings('ignore')

In [2]:
torch.set_default_dtype(torch.float64)
dtype = torch.get_default_dtype()

In [3]:
df = pd.read_csv('data.csv', header=1)
X = df.drop(columns=['thr', 'te','power'])
scaler_input = MinMaxScaler()
train_input_scaled = scaler_input.fit_transform(X)

In [4]:
current_target = 'power'
checkpoint_1 = torch.load(f"{current_target}_bestest_model.pth")
params1_1 = checkpoint_1['params']
model_power = KAN(
    layers_hidden=[8, params1_1['non'], 1],
    grid_size=params1_1['grid_size'],
    spline_order=params1_1['spline_order'],
    scale_noise=params1_1['scale_noise'],
    scale_base=params1_1['scale_base'],
    scale_spline=params1_1['scale_spline'],
    base_activation=nn.SiLU,
    grid_eps=params1_1['grid_eps'],
    grid_range=[0, 1],
    activation_scale=params1_1['activation_scale']
)
model_power.load_state_dict(checkpoint_1['model_state_dict'])

<All keys matched successfully>

In [5]:
current_target = 'thr'
checkpoint_2 = torch.load(f"{current_target}_bestest_model.pth")
params1_2 = checkpoint_2['params']
model_thr = KAN(
    layers_hidden=[8, params1_2['non'], 1],
    grid_size=params1_2['grid_size'],
    spline_order=params1_2['spline_order'],
    scale_noise=params1_2['scale_noise'],
    scale_base=params1_2['scale_base'],
    scale_spline=params1_2['scale_spline'],
    base_activation=nn.SiLU,
    grid_eps=params1_2['grid_eps'],
    grid_range=[0, 1],
    activation_scale=params1_2['activation_scale']
)
model_thr.load_state_dict(checkpoint_2['model_state_dict'])

<All keys matched successfully>

In [6]:
current_target = 'te'
checkpoint_3 = torch.load(f"{current_target}_bestest_model.pth")
params1_3 = checkpoint_3['params']
model_te = KAN(
    layers_hidden=[8, params1_3['non'], 1],
    grid_size=params1_3['grid_size'],
    spline_order=params1_3['spline_order'],
    scale_noise=params1_3['scale_noise'],
    scale_base=params1_3['scale_base'],
    scale_spline=params1_3['scale_spline'],
    base_activation=nn.SiLU,
    grid_eps=params1_3['grid_eps'],
    grid_range=[0, 1],
    activation_scale=params1_3['activation_scale']
)
model_te.load_state_dict(checkpoint_3['model_state_dict'])

<All keys matched successfully>

In [7]:
def solve():
    global powers,tes,thrs,scores
    powers,tes,thrs,scores=[],[],[],[]
    def objective(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            power = model_power(params_tensor).item()
            thr = model_thr(params_tensor).item()
            te = model_te(params_tensor).item()
        powers.append(power)
        thrs.append(thr)
        tes.append(te)
        power_scaled = (power - df.power.min()) / (df.power.max() - df.power.min())
        thr_scaled = (thr - df.thr.min()) / (df.thr.max() - df.thr.min())
        te_scaled = (te - df.te.min()) / (df.te.max() - df.te.min())
        score = thr_scaled - te_scaled-power_scaled
        scores.append(score)
        return float(score)
    opt = nlopt.opt(nlopt.LN_COBYLA, 8) 
    opt.set_lower_bounds(train_input_scaled.min(axis=0).tolist())
    opt.set_upper_bounds(train_input_scaled.max(axis=0).tolist())
    opt.set_min_objective(objective)
    def constraint_power_lower(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            power = model_power(params_tensor).item()
        return 650 - power

    def constraint_power_upper(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            power = model_power(params_tensor).item()
        return power - 660
    def constraint_thr_lower(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            thr = model_thr(params_tensor).item()
        return 7641.25 - thr

    def constraint_thr_upper(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            thr = model_thr(params_tensor).item()
        return thr - 7858.61
    def constraint_te_lower(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            te = model_te(params_tensor).item()
        return 40.23 - te

    def constraint_te_upper(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            te = model_te(params_tensor).item()
        return te - 41.84
    opt.add_inequality_constraint(constraint_te_lower)
    opt.add_inequality_constraint(constraint_te_upper)
    opt.add_inequality_constraint(constraint_thr_lower)
    opt.add_inequality_constraint(constraint_thr_upper)
    opt.add_inequality_constraint(constraint_power_lower)
    opt.add_inequality_constraint(constraint_power_upper)
    opt.set_maxeval(200)
    initial_guess = train_input_scaled[random.choice([345, 346, 347, 348, 349, 350, 351, 352, 353, 357, 
                                                  362, 364, 366, 367, 372, 373, 845, 846, 847, 848, 
                                                  861, 862, 863, 864, 865, 866, 867, 868, 869, 870, 
                                                  871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 
                                                  881, 882, 883, 884, 885, 886, 1130, 1131, 1132, 
                                                  1133, 1134, 1193, 1194, 1195, 1196, 1197, 1198, 
                                                  1199, 1200, 1260])]
    optimal_input = opt.optimize(initial_guess)
    min_score = opt.last_optimum_value()
    print(min_score)
    return powers,tes,thrs,scores,min_score,optimal_input

In [8]:
powerss, tess, thrss, scoress, min_scores, optimal_inputs = [], [], [], [], [], []

for n in range(10):
    powers_1, tes_1, thrs_1, scores_1, min_score_1, optimal_input_1 = solve()
    powerss.append(powers_1)
    tess.append(tes_1)
    thrss.append(thrs_1)
    scoress.append(scores_1)
    min_scores.append(min_score_1)
    optimal_inputs.append(optimal_input_1)

-1.6818761477685507
-1.6818887373351776
-1.6818887282576134
-1.681888737073197
-1.6752169620179935
-1.6818887369085238
-1.5730012276431773
-1.6818887105374936
-1.6818884091857205
-1.6751206288765563


In [9]:
final_results = {
    "Powers": powerss,
    "TEs": tess,
    "Thrs": thrss,
    "Scores": scoress,
    "Min Scores": min_scores,
    "Optimal Inputs": optimal_inputs
}

In [10]:
df_results = pd.DataFrame(final_results)

In [11]:
df_results.to_csv('650-660.csv',index=False)

In [12]:
def solve():
    global powers,tes,thrs,scores
    powers,tes,thrs,scores=[],[],[],[]
    def objective(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            power = model_power(params_tensor).item()
            thr = model_thr(params_tensor).item()
            te = model_te(params_tensor).item()
        powers.append(power)
        thrs.append(thr)
        tes.append(te)
        power_scaled = (power - df.power.min()) / (df.power.max() - df.power.min())
        thr_scaled = (thr - df.thr.min()) / (df.thr.max() - df.thr.min())
        te_scaled = (te - df.te.min()) / (df.te.max() - df.te.min())
        score = thr_scaled - te_scaled-power_scaled
        scores.append(score)
        return float(score)
    opt = nlopt.opt(nlopt.LN_COBYLA, 8) 
    opt.set_lower_bounds(train_input_scaled.min(axis=0).tolist())
    opt.set_upper_bounds(train_input_scaled.max(axis=0).tolist())
    opt.set_min_objective(objective)
    def constraint_power_lower(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            power = model_power(params_tensor).item()
        return 490 - power

    def constraint_power_upper(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            power = model_power(params_tensor).item()
        return power - 500
    def constraint_thr_lower(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            thr = model_thr(params_tensor).item()
        return 7487.1 - thr

    def constraint_thr_upper(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            thr = model_thr(params_tensor).item()
        return thr - 8280.68
    def constraint_te_lower(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            te = model_te(params_tensor).item()
        return 38.14 - te

    def constraint_te_upper(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            te = model_te(params_tensor).item()
        return te - 42.17
    opt.add_inequality_constraint(constraint_te_lower)
    opt.add_inequality_constraint(constraint_te_upper)
    opt.add_inequality_constraint(constraint_thr_lower)
    opt.add_inequality_constraint(constraint_thr_upper)
    opt.add_inequality_constraint(constraint_power_lower)
    opt.add_inequality_constraint(constraint_power_upper)
    opt.set_maxeval(200)
    initial_guess = train_input_scaled[random.choice([0, 12, 66, 70, 88, 94, 123, 124, 125, 165, 230, 257, 324, 330, 379, 507, 515, 538, 568, 629, 634, 915, 940, 993, 996, 1181, 1186, 1187, 1218, 1226, 1234, 1238, 1249])]
    optimal_input = opt.optimize(initial_guess)
    min_score = opt.last_optimum_value()
    print(min_score)
    return powers,tes,thrs,scores,min_score,optimal_input

powerss, tess, thrss, scoress, min_scores, optimal_inputs = [], [], [], [], [], []

for n in range(10):
    powers_1, tes_1, thrs_1, scores_1, min_score_1, optimal_input_1 = solve()
    powerss.append(powers_1)
    tess.append(tes_1)
    thrss.append(thrs_1)
    scoress.append(scores_1)
    min_scores.append(min_score_1)
    optimal_inputs.append(optimal_input_1)

final_results = {
    "Powers": powerss,
    "TEs": tess,
    "Thrs": thrss,
    "Scores": scoress,
    "Min Scores": min_scores,
    "Optimal Inputs": optimal_inputs
}
df_results = pd.DataFrame(final_results)
df_results.to_csv('490-500.csv',index=False)

-1.415012039251546
-1.4150120392922645
-1.4150120392840704
-1.4150120393829504
-1.4150120392922645
-1.415012039307321
-1.415012039330583
-1.4150120393031278
-1.415012039330583
-1.4150120392840704


In [13]:
def solve():
    global powers,tes,thrs,scores
    powers,tes,thrs,scores=[],[],[],[]
    def objective(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            power = model_power(params_tensor).item()
            thr = model_thr(params_tensor).item()
            te = model_te(params_tensor).item()
        powers.append(power)
        thrs.append(thr)
        tes.append(te)
        power_scaled = (power - df.power.min()) / (df.power.max() - df.power.min())
        thr_scaled = (thr - df.thr.min()) / (df.thr.max() - df.thr.min())
        te_scaled = (te - df.te.min()) / (df.te.max() - df.te.min())
        score = thr_scaled - te_scaled-power_scaled
        scores.append(score)
        return float(score)
    opt = nlopt.opt(nlopt.LN_COBYLA, 8) 
    opt.set_lower_bounds(train_input_scaled.min(axis=0).tolist())
    opt.set_upper_bounds(train_input_scaled.max(axis=0).tolist())
    opt.set_min_objective(objective)
    def constraint_power_lower(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            power = model_power(params_tensor).item()
        return 350 - power

    def constraint_power_upper(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            power = model_power(params_tensor).item()
        return power - 360
    def constraint_thr_lower(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            thr = model_thr(params_tensor).item()
        return 7972.48 - thr

    def constraint_thr_upper(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            thr = model_thr(params_tensor).item()
        return thr - 8205.94
    def constraint_te_lower(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            te = model_te(params_tensor).item()
        return 38.0 - te

    def constraint_te_upper(params, grad):
        params_tensor = torch.tensor(params, dtype=dtype)
        with torch.no_grad():
            te = model_te(params_tensor).item()
        return te - 38.81
    opt.add_inequality_constraint(constraint_te_lower)
    opt.add_inequality_constraint(constraint_te_upper)
    opt.add_inequality_constraint(constraint_thr_lower)
    opt.add_inequality_constraint(constraint_thr_upper)
    opt.add_inequality_constraint(constraint_power_lower)
    opt.add_inequality_constraint(constraint_power_upper)
    opt.set_maxeval(200)
    initial_guess = train_input_scaled[random.choice([255, 256, 264, 265, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 518, 519, 544, 734])]
    optimal_input = opt.optimize(initial_guess)
    min_score = opt.last_optimum_value()
    print(min_score)
    return powers,tes,thrs,scores,min_score,optimal_input

powerss, tess, thrss, scoress, min_scores, optimal_inputs = [], [], [], [], [], []

for n in range(10):
    powers_1, tes_1, thrs_1, scores_1, min_score_1, optimal_input_1 = solve()
    powerss.append(powers_1)
    tess.append(tes_1)
    thrss.append(thrs_1)
    scoress.append(scores_1)
    min_scores.append(min_score_1)
    optimal_inputs.append(optimal_input_1)

final_results = {
    "Powers": powerss,
    "TEs": tess,
    "Thrs": thrss,
    "Scores": scoress,
    "Min Scores": min_scores,
    "Optimal Inputs": optimal_inputs
}
df_results = pd.DataFrame(final_results)
df_results.to_csv('350-360.csv',index=False)

0.5418274506646916
0.42672298000856324
0.4267098848656585
0.426709880424734
0.426709864309734
0.4267098848656585
0.4267098848656585
0.426709880424734
0.426709864309734
0.42670988487953737
